In [ ]:
from pathlib import Path
import data

import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

In [ ]:
# --- Extract data from local files

n = 10000
data_path = Path('.', 'MNIST')

x_train, y_train = data.get_train_data(data_path, n)
x_test, y_test = data.get_test_data(data_path, n)
x_train_img = x_train.reshape(n, 1, 28, 28)
x_test_img = x_test.reshape(n, 1, 28, 28)

data.show_sample(x_train_img, y_train, 10)

In [ ]:
# --- Standardize the data

def standardized(data):
    return (data - data.mean()) / data.std()

x_train = standardized(x_train)
x_test = standardized(x_test)
x_train_img = standardized(x_train_img)
x_test_img = standardized(x_test_img)

In [ ]:
# --- Create the model

# import mlp_classifier
# model = mlp_classifier.FullyConnectedClassifier()

import cnn_classifier
model = cnn_classifier.ConvolutionalClassifier()

In [ ]:
# --- Test the model before training

with torch.no_grad():
    # y_test_pred = model(x_test).argmax(dim=1)
    y_test_pred = model(x_test_img).argmax(dim=1)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)

In [ ]:
# --- Train the model

class TrainData(Dataset):
    def __init__(self, x_train, y_train):
        assert x_train.size(0) == y_train.size(0)
        self.x_train = x_train
        self.y_train = y_train

    def __len__(self):
        return self.x_train.size(0)

    def __getitem__(self, idx):
        return self.x_train[idx], self.y_train[idx]

# epoch_values, loss_values = mlp_classifier.train(
#     model,
#     TrainData(x_train, y_train),
#     nb_epochs=100,
#     batch_size=64,
#     learning_rate=0.01
# )

epoch_values, loss_values = cnn_classifier.train(
    model,
    TrainData(x_train_img, y_train),
    nb_epochs=100,
    batch_size=64,
    learning_rate=0.01
)


plt.title("Loss over epochs")
plt.plot(epoch_values, loss_values)
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# --- Test the model after training

with torch.no_grad():
    # y_test_pred = model(x_test).argmax(dim=1)
    y_test_pred = model(x_test_img).argmax(dim=1)

accuracy = accuracy_score(y_test, y_test_pred)
print(f"{accuracy = }")

ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred)